# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '4505346f-79e9-48b2-9794-b785c33af8d7', 'created_at': '2025-08-12T07:40:37.745504+00:00', 'updated_at': '2025-08-12T07:40:37.745507+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
4505346f-79e9-48b2-9794-b785c33af8d7


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [3]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [4]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [5]:
input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드 생성을 도와드리겠습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?


In [6]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            data = get_data(chunk)
            print(data['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋아요! 제주도 여행에 대한 슬라이드를 생성하기 위해 추가로 몇 가지 정보를 알고 싶어요.

1. 슬라이드의 대상 청중은 누구인가요? 예를 들어, 친구, 가족, 고객 등.
2. 슬라이드에 포함하고 싶은 주요 포인트나 내용은 어떤 것들인가요? 예를 들어, 유명 관광지, 맛집, 여행 팁 등.
3. 슬라이드 페이지 수나 디자인 스타일에 대한 특별한 요구사항이 있을까요?


In [7]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):

    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        msg_id = data['messages'][-1]['id']

        print(data['messages'][-1]['content'])
    elif filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)


##########  AI 메시지  ##########

##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도를 여행할 때 유용한 기본 정보를 제공합니다. 여행 계획을 세우는 데 도움이 될 장소들, 교통편, 숙소 선택 팁 등을 포함합니다. 
3. 제주도 여행 - 제주도의 대표 관광 명소들을 소개합니다. 성산일출봉, 한라산, 섭지코지 등 자연 경관과 여행지에서 꼭 가보아야 할 명소들을 강조합니다. 
4. 제주도 여행 - 제주도의 전통 문화를 체험할 수 있는 다양한 장소와 활동을 살펴봅니다. 한복 렌탈과 전통 놀이 체험, 해녀 문화 체험, 전통 마을 방문 등을 포함합니다. 
5. 제주도 여행 - 제주도의 풍요로운 미식 경험을 위한 맛집을 추천합니다. 현지인 추천 맛집과 독특한 제주 요리, 해산물 음식점 등을 중심으로 소개합니다. 
6. 제주도 여행 - 제주도에서 즐길 수 있는 수많은 액티비티를 소개합니다. 올레길 트래킹, 스쿠버다이빙, 패러글라이딩, 요트 투어 등 다양한 체험거리를 포함합니다. 

##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  AI 메시지  ##########
제주도 여행을 주제로 한 슬라이드는 다음과 같은 내용으로 구성될 예정입니다.

1. **제주도 여행 기본 정보**: 여행 계획을 세울 때 유용한 장소들, 교통편, 숙소 선택 팁 등을 포함합니다.
2. **제주도의 대표 관광 명소*

In [8]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

In [9]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [10]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
안녕하세요! 슬라이드 생성을 도와드리겠습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?
##########  AI 메시지  ##########
좋아요! 제주도 여행에 대한 슬라이드를 생성하기 위해 추가로 몇 가지 정보를 알고 싶어요.

1. 슬라이드의 대상 청중은 누구인가요? 예를 들어, 친구, 가족, 고객 등.
2. 슬라이드에 포함하고 싶은 주요 포인트나 내용은 어떤 것들인가요? 예를 들어, 유명 관광지, 맛집, 여행 팁 등.
3. 슬라이드 페이지 수나 디자인 스타일에 대한 특별한 요구사항이 있을까요?
##########  AI 메시지  ##########

##########  AI 메시지  ##########
Successfully transferred to slide_create_agent
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도를 여행할 때 유용한 기본 정보를 제공합니다. 여행 계획을 세우는 데 도움이 될 장소들, 교통편, 숙소 선택 팁 등을 포함합니다. 
3. 제주도 여행 - 제주도의 대표 관광 명소들을 소개합니다. 성산일출봉, 한라산, 섭지코지 등 자연 경관과 여행지에서 꼭 가보아야 할 명소들을 강조합니다. 
4. 제주도 여행 - 제주도의 전통 문화를 체험할 수 있는 다양한 장소와 활동을 살펴봅니다. 한복 렌탈과 전통 놀이 체험, 해녀 문화 체험, 전통 마을 방문 등을 포함합니다. 
5. 제주도 여행 - 제주도의 풍요로운 미식 경험을 위한 맛집을 추천합니다. 현지인 추천 맛집과 독특한 제주 요리, 해산물 음식점 등을 중심으로 소개합니다. 
6. 제주도 여행 - 제주도에서 즐길 수 있는 수많은 액티비티를 소개합니다. 올레길 트래킹, 스쿠버다이빙, 패러글라이딩, 요트 투어 등 다양한 체험거리를 포함합니

In [11]:
len(state["values"]["slides"])

14

In [12]:
state["values"]["slides"]

[{'topic': '제주도 여행',
  'name': '타이틀 슬라이드',
  'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'content': '## 타이틀 슬라이드\n\nJeju Island, often referred to as the "Hawaii of Korea," offers an enchanting escape with its breathtaking landscapes and unique cultural experiences. The island\'s natural beauty, from the iconic Hallasan Mountain to the exotic beaches and lush forests, provides a diverse range of attractions for tourists. Whether you\'re interested in hiking along the famous Olle trails, exploring the stunning Jeongbang Waterfall, or savoring local delicacies, Jeju Island caters to all types of travelers. In addition to its natural wonders, Jeju\'s vibrant arts scene and traditional markets give insight into the rich history and culture of the region, making it a comprehensive destination for those seeking both relaxation and adventure [1][2].\n\nJeju’s tourism is further enhanced by its status as a UNESCO World Heritage Site, owing to the significant geological and ecological feature

In [13]:
for s in state["values"]["slides"]:
    print(s["idx"])

KeyError: 'idx'

In [ ]:
state["values"]['sections']